In [28]:
import local_models.local_models
import local_models.algorithms
import local_models.utils
import local_models.linear_projections
import local_models.loggin
import local_models.TLS_models
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
import sklearn.cluster
from importlib import reload
from ml_battery.utils import cmap
import matplotlib as mpl
import sklearn.datasets
import sklearn.decomposition
import logging
import ml_battery.log
import time
import os
import mayavi
import mayavi.mlab
import string
import subprocess
import functools
#on headless systems, tmux: "Xvfb :1 -screen 0 1280x1024x24 -auth localhost", then "export DISPLAY=:1" in the jupyter tmux
mayavi.mlab.options.offscreen = True



logger = logging.getLogger(__name__)

#reload(local_models.local_models)
#reload(lm)
#reload(local_models.loggin)
#reload(local_models.TLS_models)
np.warnings.filterwarnings('ignore')


In [2]:
def import_shit():
    import local_models.local_models
    import local_models.algorithms
    import local_models.utils
    import local_models.linear_projections
    import local_models.loggin
    import local_models.TLS_models
    import numpy as np
    import logging
    import string
    import ml_battery.log


    logger = logging.getLogger(__name__)

    #reload(local_models.local_models)
    #reload(lm)
    #reload(local_models.loggin)
    #reload(local_models.TLS_models)
    np.warnings.filterwarnings('ignore')
    return logger

In [3]:
FRESH=True

In [4]:
kernel_names = {
    local_models.local_models.GaussianKernel: 'gaussian',
    local_models.local_models.TriCubeKernel: 'tricube'
}

In [5]:
mpl.rcParams['figure.figsize'] = [8.0, 8.0]

In [6]:
data = np.loadtxt("../data/cmu_campus/oakland_part3_an_training.xyz_label_conf")[:,:3]

In [7]:
(data.shape[0]*100)**(1/3)

154.57322768137988

In [12]:
KERNEL=local_models.local_models.GaussianKernel
RUN = 5
BANDWIDTHS = np.linspace(0.2,14.6,29)

project_dir = "../data/3dscene_filler_{}_{:03d}".format(kernel_names[KERNEL], RUN)

os.makedirs(project_dir, exist_ok=1)

In [13]:
print(BANDWIDTHS[:-1] - BANDWIDTHS[1:])


[-0.51428571 -0.51428571 -0.51428571 -0.51428571 -0.51428571 -0.51428571
 -0.51428571 -0.51428571 -0.51428571 -0.51428571 -0.51428571 -0.51428571
 -0.51428571 -0.51428571 -0.51428571 -0.51428571 -0.51428571 -0.51428571
 -0.51428571 -0.51428571 -0.51428571 -0.51428571 -0.51428571 -0.51428571
 -0.51428571 -0.51428571 -0.51428571 -0.51428571]


In [14]:
mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(800, 800))
#mayavi.mlab.surf(grid[0], grid[1], kde_wireframe/z_scale, colormap='Greys', opacity=1)
#nodes = mayavi.mlab.points3d(converged[:,0], converged[:,1], converged[:,2], scale_mode='scalar', color=(1,0,0))
#nodes.mlab_source.dataset.point_data.scalars = np.ones(converged.shape[0])*0.1
bunodes = mayavi.mlab.points3d(data[:,0], data[:,1], data[:,2], scale_mode='scalar', color=(0,1,0))
bunodes.mlab_source.dataset.point_data.scalars = np.ones(data.shape[0])*0.05


#mayavi.mlab.axes()

#mayavi.mlab.view(views[0][1],views[0][0])
data_avg = np.average(data, axis=1)
mayavi.mlab.view(azimuth=180, elevation=80, distance=1*np.average(data_avg), focalpoint=(data_avg[0], data_avg[1], data_avg[2]))
mayavi.mlab.move(forward=None, right=0.1*data_avg[1], up=-0.02*data_avg[0])
title = "data"
mayavi.mlab.savefig(os.path.join(project_dir, "{}.png".format(title)))
mayavi.mlab.clf()

In [15]:
#visual = pcl.pcl_visualization.CloudViewing()
#visual.ShowMonochromeCloud(room)

In [16]:
linear_models = local_models.local_models.LocalModels(local_models.TLS_models.LinearODR_mD(2))
linear_models.fit(data)

In [17]:
avg_pt_dist = np.average(linear_models.index.query(data, k=2)[0][:,1])

In [18]:
avg_pt_dist

0.13714420516231735

In [19]:
range_pct = 0.2
grid_steps = 50
data_mins, data_maxes, data_ranges = local_models.linear_projections.min_max_range(data)
graph_bounds = local_models.linear_projections.sane_graph_bounds(data_mins, data_maxes, data_ranges, range_pct)

grid = local_models.utils.get_global_grid(data, outside_range_pct=range_pct, n_steps=grid_steps)
gridder = local_models.utils.Grid2Vec()
grid = gridder.fit_transform(grid)

In [20]:
pth = os.path.join(project_dir, "filler")
os.makedirs(pth, exist_ok=1)

In [21]:
def converge_n_save(bandwidth, i, data, pth):
    logger = import_shit()
    logger.info("stuff!")
    for converged in local_models.algorithms.local_tls_shift_till_convergence(linear_models, data, tol=1e-8,
                                         kernel=KERNEL(bandwidth),
                                         report=False):
        pass
    logger.info("batch {:05d} complete".format(i))
    save_dat(i, pth, converged)

def save_dat(i, pth, dat):
    np.savetxt(os.path.join(pth, "converged{:08d}.dat".format(i)), dat)

def converge_n_return(bandwidth, i, data):
    logger = import_shit()
    logger.info("stuff!")
    for converged in local_models.algorithms.local_tls_shift_till_convergence(linear_models, data, tol=1e-8,
                                         kernel=KERNEL(bandwidth),
                                         report=False):
        pass
    logger.info("batch {:05d} complete".format(i))
    return converged

def batcher(data, size):
    for i in range(0, data.shape[0], size):
        batch = data[i:i+size]
        yield batch

In [ ]:
BATCH_SIZE = 1000

convergedededs = []
for BANDWIDTH in BANDWIDTHS:
    bandwidth_path = os.path.join(pth, "b{:08.02f}".format(BANDWIDTH))
    os.makedirs(bandwidth_path, exist_ok=1)
    if FRESH:
        from joblib import Parallel, delayed
        Parallel(n_jobs=32)(delayed(converge_n_save)(BANDWIDTH, i, batch, bandwidth_path) for i, batch in enumerate(batcher(grid, BATCH_SIZE)))
    elif FRESH_DASK:
        import dask.distributed
        client = dask.distributed.Client("localhost:8786")
        futures = client.map(lambda *args: converge_n_return(BANDWIDTH, *args), enumerate(batcher(grid, BATCH_SIZE))) 
        for future in dask.distributed.as_completed(futures):
            save_dat()


In [ ]:
def mean_center(data, weights=None):
    return data - np.average(data, axis=0,weights=weights)

In [25]:
def load_converged_data(pth, bandwidth):
    bandwidth_path = os.path.join(pth, "b{:08.02f}".format(bandwidth))
    convergededs = []
    for dat in sorted(os.listdir(bandwidth_path)):
        convergededs.append(np.loadtxt(os.path.join(bandwidth_path, dat)))
    return np.concatenate(convergededs, axis=0)

def plt_grid(fig, grid, data_avg, data_std):
    nodes = mayavi.mlab.points3d(grid[:,0], grid[:,1], grid[:,2], 
                                 scale_mode='scalar', colormap='gist_earth', figure=fig)
    nodes.glyph.scale_mode = 'scale_by_vector'
    nodes.mlab_source.dataset.point_data.vectors = np.ones((grid.shape[0],3))*(np.average(data_std)/400)
    nodes.mlab_source.dataset.point_data.scalars = (grid[:,1] - (data_avg[1]-3*data_std[1]))/(6*data_std[1])
    return nodes

def plt_data(fig, data):
    nodes = mayavi.mlab.points3d(data[:,0], data[:,1], data[:,2], scale_mode='scalar', color=(0,1,0), figure=fig)
    nodes.mlab_source.dataset.point_data.scalars = np.ones(data.shape[0])*0.05
    return nodes

def normalize_view(fig, data_avg, data_std):
    mayavi.mlab.view(azimuth=180, elevation=80, distance=1*np.average(data_avg), focalpoint=(data_avg[0], data_avg[1], data_avg[2]), figure=fig)
    mayavi.mlab.move(forward=None, right=0.1*data_avg[1], up=-0.02*data_avg[0], figure=fig)
    
def plt_and_save(data, grid, bandwidth, pth):
    data_avg = np.average(data, axis=0)
    data_std = np.std(data, axis=0)
    figure = mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(1000, 500))
    converged_nodes = plt_grid(figure, grid, data_avg, data_std)
    data_nodes = plt_data(figure, data)
    normalize_view(figure, data_avg, data_std)
    mayavi.mlab.savefig(pth)
    mayavi.mlab.close(figure)
    
def serialize_plt(pth):
    import zlib
    with open(pth, 'rb') as f:
        dat = f.read()
    return zlib.compress(dat)
   
def deserialize_plt(dat, pth):
    import zlib
    with open(pth, 'wb') as f:
        f.write(zlib.decompress(dat))
    return pth

def distributed_plt_and_save(data, grid, bandwidth):
    import numpy as np
    import mayavi
    import mayavi.mlab
    import string
    import os
    #on headless systems, tmux: "Xvfb :1 -screen 0 1280x1024x24 -auth localhost", then "export DISPLAY=:1" in the jupyter tmux
    mayavi.mlab.options.offscreen = True
    
    unique_id = "".join(np.random.choice(list(string.ascii_lowercase), replace=True, size=20))
    pth = "/ramfs/{}.png".format(unique_id)
    try:
        plt_and_save(data, grid, bandwidth, pth)
        result = serialize_plt(pth)
    except:
        os.remove(pth)

In [30]:
for i in range(len(BANDWIDTHS)):
    title = "converged_data_b{:08.02f}".format(BANDWIDTHS[i])
    img_pth = os.path.join(pth, "{}.png".format(title))
    grid = load_converged_data(pth, BANDWIDTHS[i])
    plt_and_save(data, grid, bandwidth, img_pth)

TypeError: move() got an unexpected keyword argument 'figure'

In [44]:
mayavi.mlab.close(all=True)

In [ ]:
bandwidths = [BANDWIDTH]
converged = np.concatenate(convergededs)
for i, bandwidth in enumerate(bandwidths):
    data_avg = np.average(converged, axis=0)
    data_std = np.std(converged, axis=0)
    figure = mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(1000, 500))
    #mayavi.mlab.surf(grid[0], grid[1], kde_wireframe/z_scale, colormap='Greys', opacity=1)
    nodes = mayavi.mlab.triangular_mesh(converged[:,0], converged[:,1], converged[:,2], 
                                 scale_mode='scalar', colormap='gist_earth')
    nodes.glyph.scale_mode = 'scale_by_vector'
    nodes.mlab_source.dataset.point_data.vectors = np.ones((converged.shape[0],3))*(np.average(data_std)/400)
    nodes.mlab_source.dataset.point_data.scalars = (converged[:,1] - (data_avg[1]-3*data_std[1]))/(6*data_std[1])
    #bunodes = mayavi.mlab.points3d(data[:,0], data[:,1], data[:,2], scale_mode='scalar', color=(0,1,0))
    #bunodes.mlab_source.dataset.point_data.scalars = np.ones(data.shape[0])*0.1


    #mayavi.mlab.axes()

    #mayavi.mlab.view(views[0][1],views[0][0])
    mayavi.mlab.view(azimuth=180, elevation=80, distance=1*np.average(data_avg), focalpoint=(data_avg[0], data_avg[1], data_avg[2]))
    mayavi.mlab.move(forward=None, right=0.1*data_avg[1], up=-0.02*data_avg[0])
    title = "converged_data_b{:08.02f}".format(bandwidth)
    mayavi.mlab.savefig(os.path.join(pth, "{}.png".format(title)))
    mayavi.mlab.close(figure)

In [ ]:
mayavi.mlab.close(all=True)

In [ ]:
range_pct = 0.2
grid_steps = 20
data_mins, data_maxes, data_ranges = local_models.linear_projections.min_max_range(data)
graph_bounds = local_models.linear_projections.sane_graph_bounds(data_mins, data_maxes, data_ranges, range_pct)

grid_limits = tuple(map(lambda i: slice(graph_bounds[0,i], graph_bounds[1,i], grid_steps*1j), range(graph_bounds.shape[1])))
grid = np.mgrid[grid_limits]
gridder = local_models.utils.Grid2Vec()
grid = gridder.fit_transform(grid)

In [ ]:
for converged2 in local_models.algorithms.local_tls_shift_till_convergence(linear_models, grid, tol=1e-10,
                                     kernel=KERNEL(bandwidth),
                                     report=False):
    pass

In [ ]:
converged2

In [ ]:
cloud = pcl.PointCloud()
cloud.from_array(converged2.astype('float32'))
#visual = pcl.pcl_visualization.CloudViewing()
#visual.ShowMonochromeCloud(cloud)

In [ ]:
mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(800, 800))
#mayavi.mlab.surf(grid[0], grid[1], kde_wireframe/z_scale, colormap='Greys', opacity=1)
nodes = mayavi.mlab.points3d(converged2[:,0], converged2[:,1], converged2[:,2], scale_mode='scalar', color=(1,0,0))
nodes.mlab_source.dataset.point_data.scalars = np.ones(converged2.shape[0])*0.1
bunodes = mayavi.mlab.points3d(data[:,0], data[:,1], data[:,2], scale_mode='scalar', color=(0,1,0))
bunodes.mlab_source.dataset.point_data.scalars = np.ones(data.shape[0])*0.1


#mayavi.mlab.axes()

#mayavi.mlab.view(views[0][1],views[0][0])
data_avg = np.average(converged2, axis=1)
mayavi.mlab.view(azimuth=0, elevation=30, distance=1, focalpoint=(data_avg[0], data_avg[1], data_avg[2]))
title = "converged_g000{:05d}".format(grid_steps)
mayavi.mlab.savefig(os.path.join(project_dir, "{}.png".format(title)))
mayavi.mlab.clf()